In [16]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as tfs

import matplotlib.pyplot as plt
import numpy as np
import random
import json
import time

import math
from models import LogisticRegressor, Convolutional
from run_pipeline import main

In [17]:
N_EPOCHS = 2000
F_RANGE = (2, 40)
W_RANGE = (-5, 0)
W_CENTER = 10

In [18]:
n_features = abs(F_RANGE[1] - F_RANGE[0])
win_dim = abs(W_RANGE[1] - W_RANGE[0])

In [19]:
train_data = []
test_data = []
n_videos = 0

n_true, n_false = 0, 0

def construct_tensors(matrix, state):
    w_range = (W_RANGE[0] + W_CENTER, W_CENTER + W_RANGE[1])
    matrix = np.abs(np.array(matrix))

    # Prepare vector & matrix for regressor & convolutional
    r_vector = matrix[W_CENTER, F_RANGE[0]:F_RANGE[1]].flatten()
    c_matrix = matrix[w_range[0]:w_range[1], F_RANGE[0]:F_RANGE[1]]

    x0 = torch.FloatTensor(r_vector)
    x1 = torch.FloatTensor([c_matrix])
    y = torch.tensor(state)
    return (x0, x1, y)

with open('data/mfccs_40_4096_1024.json') as f:
    json_data = json.load(f)
    n_videos = len(json_data)

    # Loop over all videos in dataset
    for i, video in enumerate(json_data):
        video_data = json_data[video]

        for data in video_data:
            matrix, state = data['m'], data['s']
            state_condition = n_true > n_false if state is 0 else True

            if matrix != [] and state_condition:
                n_true += state
                n_false += 1 - state  
                if i < 8:
                    train_data.append(construct_tensors(matrix, state))
                else:
                    test_data.append(construct_tensors(matrix, state))

print(f'Extracted {len(train_data)} train points and {len(test_data)} test points from {n_videos} videos')
print(f'n_true: {n_true}, n_false: {n_false}')

<>:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-19-4e4012a1466e>:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
  state_condition = n_true > n_false if state is 0 else True


FileNotFoundError: [Errno 2] No such file or directory: 'data/mfccs_40_4096_1024.json'

In [20]:
print(train_data[0][0].shape)
print(train_data[0][1].shape)

IndexError: list index out of range

In [21]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=64,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size=64,
                                           shuffle=True)

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [22]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print(f'Device: {device}')

Device: cuda:0


In [23]:
regressor, conv = LogisticRegressor(n_features, 1, 2).to(device), Convolutional(n_features, win_dim, 2).to(device)
print(f'n_params={sum(p.numel() for p in regressor.parameters())}')
print(f'n_params={sum(p.numel() for p in conv.parameters())}')

n_params=1662
n_params=12027


In [24]:
optis = [optim.SGD(regressor.parameters(), lr=0.001), optim.SGD(conv.parameters(), lr=0.001)]
crits = [nn.CrossEntropyLoss(), nn.CrossEntropyLoss()]

In [25]:
def accuracy(model, data, inds):
    x, y = inds
    correct, total = 0, 0
    for xxy in data:
        y_hat = model(xxy[x].to(device))
        _, predicted = torch.max(y_hat.data, 1)
        
        total += xxy[y].size(0)
        correct += (predicted.cpu() == xxy[y]).sum()

    return 100 * int(correct) / total

def confusion(model, data, inds):
    x, y = inds
    matrix = np.array([[0, 0], [0, 0]])
    exampl = [[[], []], [], []]
    for xxy in data:
        y_hat = model(xxy[x].to(device))
        _, predicted = torch.max(y_hat.data, 1)
        predicted = predicted.cpu()

        matrix[0, 0] += torch.sum((xxy[y] == 1) & (predicted == 1))
        matrix[0, 1] += torch.sum((xxy[y] == 0) & (predicted == 1))
        matrix[1, 0] += torch.sum((xxy[y] == 1) & (predicted == 0))
        matrix[1, 1] += torch.sum((xxy[y] == 0) & (predicted == 0))

    return matrix

def recall(model, data, inds):
    x, y = inds
    tp, fn = 0, 0
    for xxy in data:
        y_hat = model(xxy[x].to(device))
        _, predicted = torch.max(y_hat.data, 1)

        tp += torch.sum((xxy[y] == 1) & (predicted.cpu() == 1))
        fn += torch.sum((xxy[y] == 1) & (predicted.cpu() == 0))
    return int(tp) / (int(tp) + int(fn))


In [26]:
log_int = 100
accuracy_histories = [[], []]
n_iters = 0
start = time.time()

In [27]:
for epoch in range(1, N_EPOCHS + 1):
    for i, (x0, x1, y) in enumerate(train_loader):
        # Clear gradients
        optis[0].zero_grad()
        optis[1].zero_grad()

        # Make prediction and calculate loss
        y_hat0 = regressor(x0.to(device)).cpu()
        y_hat1 = conv(x1.to(device)).cpu()

        loss0 = crits[0](y_hat0, y)
        loss1 = crits[1](y_hat1, y)

        # Calculate new gradients and optimize
        loss0.backward()
        loss1.backward()

        optis[0].step()
        optis[1].step()

        if n_iters % log_int == 0:
            acc0 = accuracy(regressor, train_loader, (0, 2))
            acc1 = accuracy(regressor, test_loader, (0, 2))
            acc2 = accuracy(conv, train_loader, (1, 2))
            acc3 = accuracy(conv, test_loader, (1, 2))

            accuracy_histories[0].append(acc1)
            accuracy_histories[1].append(acc3)
            
            fepoch = 'Epoch: \033[92m{}\033[0m/\033[92m{}\033[0m'.format(epoch, N_EPOCHS)
            fiters = 'Iter: \033[92m{}\033[0m'.format(n_iters)
            facc0 = 'Acc0: \033[92m{:.2f}%\033[0m, \033[92m{:.2f}%\033[0m'.format(acc0, acc1)
            facc1 = 'Acc1: \033[92m{:.2f}%\033[0m, \033[92m{:.2f}%\033[0m'.format(acc2, acc3)

            fptime = 'Time: \033[92m{:.0f}\033[0ms'.format(time.time() - start)
            print('\r{}\t | {}\t | {}\t | {}\t | {}\t |'.format(fepoch, fiters, facc0, facc1, fptime), end="")
        n_iters += 1

NameError: name 'train_loader' is not defined

In [28]:
regressor.eval()
conv.eval()

print(f'REGRESSOR CONFUSION MATRIX:\n {confusion(regressor, test_loader, (0, 2))}')
print(f'CONVOLUTIONAL CONFUSION MATRIX:\n {confusion(conv, test_loader, (1, 2))}')

%matplotlib inline
x0, x1 = np.arange(len(accuracy_histories[0])) * log_int, np.arange(len(accuracy_histories[1])) * log_int
f0, f1 = np.polyfit(x0, accuracy_histories[0], 5), np.polyfit(x1, accuracy_histories[1], 5)
f0, f1 = np.poly1d(f0), np.poly1d(f1)

plt.scatter(x0, accuracy_histories[0], color='r', alpha=0.05)
plt.scatter(x1, accuracy_histories[1], color='b', alpha=0.05)
plt.plot(x0, f0(x0), color='r', label='regressor')
plt.plot(x1, f1(x1), color='b', label='convolutional')

plt.legend()
plt.show()

NameError: name 'test_loader' is not defined

In [29]:
torch.save(regressor.state_dict(), 'data/models/regressor.pkl')
torch.save(conv.state_dict(), 'data/models/convolutional.pkl')

In [30]:
fig = plt.figure()
for i in range(10):
    print(test_data[i + 110][2])
    plt.imshow(np.rot90(test_data[i + 110][1][0]))
    plt.show()

IndexError: list index out of range